### Data Cleaning and Preperation

In [ ]:
#importing data
import pandas as pd

errordf = pd.read_csv("erroredlinks.modeling.csv")
errordf.head

c:\Users\Ksell\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


<bound method NDFrame.head of                                                      url Found Links  \
0      http://0.olderwiseraod.net/about/faculty-and-s...       ERROR   
1                 http://1065lakesfm.com/bloodmobile-18/       ERROR   
2      http://111.68.103.26//journals/index.php/pujm/...       ERROR   
3      http://150-degree.com/bilder/ebook/download-en...       ERROR   
4      http://150-degree.com/bilder/ebook/download-pe...       ERROR   
...                                                  ...         ...   
76041  https://zizzmedia.com/netizens-hilarious-take-...       ERROR   
76042                              https://zmplelux.com/       ERROR   
76043  https://zoebleuvert.com/products/chloe-denim-j...       ERROR   
76044  https://zqhgz.com/tailored-for-tomorrow-the-pi...       ERROR   
76045  https://zulrish.com/2024/01/23/gastric-balloon...       ERROR   

                                                 content  
0      faculty and staff  cardinal mooney cath

In [ ]:
#getting rid of special characters and lower caseing content
pattern = r'[^a-zA-Z0-9\s]'
errordf['content'] = errordf['content'].str.replace(pattern,'',regex=True)
# Remove newline characters
errordf['content'] = errordf['content'].str.replace(r'\n', '', regex=True)
errordf = errordf.applymap(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\Ksell\AppData\Local\Temp\ipykernel_33704\1908475982.py:6: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [ ]:
#key words for narrowing down errored links

keywords = ['census', 'uscb']

# Function to check if any keywords are in the content
def contains_keywords(content, keywords):
    return any(keyword in content for keyword in keywords)

# Apply the function to create a new column
errordf['Keyword'] = errordf['content'].apply(lambda x: "true" if contains_keywords(x, keywords) else "false")

print(errordf)

                                                     url Found Links  \
0      http://0.olderwiseraod.net/about/faculty-and-s...       error   
1                 http://1065lakesfm.com/bloodmobile-18/       error   
2      http://111.68.103.26//journals/index.php/pujm/...       error   
3      http://150-degree.com/bilder/ebook/download-en...       error   
4      http://150-degree.com/bilder/ebook/download-pe...       error   
...                                                  ...         ...   
76041  https://zizzmedia.com/netizens-hilarious-take-...       error   
76042                              https://zmplelux.com/       error   
76043  https://zoebleuvert.com/products/chloe-denim-j...       error   
76044  https://zqhgz.com/tailored-for-tomorrow-the-pi...       error   
76045  https://zulrish.com/2024/01/23/gastric-balloon...       error   

                                                 content Keyword  
0      faculty and staff  cardinal mooney catholic hi...   false  
1

In [ ]:
errordf = errordf[errordf['Keyword']=='true']
errordf.head

<bound method NDFrame.head of                                                      url Found Links  \
3      http://150-degree.com/bilder/ebook/download-en...       error   
4      http://150-degree.com/bilder/ebook/download-pe...       error   
9      http://3er-schmiede.de/wellrad/plugins/wellrad...       error   
124    http://automatedgenealogy.com/census/view.jsp?...       error   
125    http://automatedgenealogy.com/census11/view.js...       error   
...                                                  ...         ...   
75647    https://www.thymio.org/sour-gummies-a98662.html       error   
75763  https://www.universetoday.com/tag/protoplaneta...       error   
75821  https://www.visionofbritain.org.uk/unit/102274...       error   
75863  https://www.wecdsb.on.ca/cms/one.aspx?portalid...       error   
75873  https://www.wellnessdentalllc.com/for-healthy-...       error   

                                                 content Keyword  
3      download encyclopedic dictionar

In [ ]:
errorlinks = errordf['url'].tolist()

len(errorlinks)

1720

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# List of URLs to process
urls = errorlinks

# Compile the regex pattern
pattern = re.compile(r'.census.gov.*')

# Initialize an empty list to store all data
all_data = []

# Iterate over each URL in the list
for url in urls:
    try:
        # Attempt to fetch the content from the URL
        response = requests.get(url, timeout=5)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Parse the content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all links matching the pattern
        links = [link.get('href') for link in soup.find_all('a', href=pattern)]

        if links:
            all_data.extend([(url, link) for link in links])
        else:
            all_data.append((url, 'NONE'))

    except requests.exceptions.RequestException as e:
        # Handle any exception raised during the request
        error_code = e.response.status_code if e.response else 'No Response'
        error_message = str(e)
        print(f"Failed to scrape {url}: {error_message} (Error Code: {error_code})")
        # Add a failure entry to the data
        all_data.append((url, f'ERROR: {error_code}', error_message))

# Create a DataFrame with the seed links and the found links
scrapedlinks = pd.DataFrame(all_data, columns=['url', 'Found Links', 'Error Message'])
print(scrapedlinks)

Failed to scrape http://150-degree.com/bilder/ebook/download-encyclopedic-dictionary-of-landscape-and-urban-planning-2010.php: 404 Client Error: Not Found for url: http://150-degree.com/bilder/ebook/download-encyclopedic-dictionary-of-landscape-and-urban-planning-2010.php (Error Code: No Response)
Failed to scrape http://150-degree.com/bilder/ebook/download-permanent-habit-control%3a-practitioner%e2%80%99s-guide-to-using-hypnosis-and-other-alternative-health-strategies.php: 404 Client Error: Not Found for url: http://150-degree.com/bilder/ebook/download-permanent-habit-control%3A-practitioner%E2%80%99s-guide-to-using-hypnosis-and-other-alternative-health-strategies.php (Error Code: No Response)
Failed to scrape http://3er-schmiede.de/wellrad/plugins/wellrad/ebook.php?q=online-der-schluchtwald-und-der-bach-eschenwald-1952.html: 404 Client Error: Not Found for url: http://3er-schmiede.de/wellrad/plugins/wellrad/ebook.php?q=online-der-schluchtwald-und-der-bach-eschenwald-1952.html (Error 

In [ ]:
#output links from web scraper so we don't have to scrape them repeatedly
scrapedlinks.to_excel('errorcodes.xlsx', index=False)

In [ ]:
#pulling in files
elinks = pd.read_excel("errorcodes.xlsx")
elinks.head

<bound method NDFrame.head of                                                     url         Found Links  \
0     http://150-degree.com/bilder/ebook/download-en...  ERROR: No Response   
1     http://150-degree.com/bilder/ebook/download-pe...  ERROR: No Response   
2     http://3er-schmiede.de/wellrad/plugins/wellrad...  ERROR: No Response   
3     http://automatedgenealogy.com/census/view.jsp?...  ERROR: No Response   
4     http://automatedgenealogy.com/census11/view.js...  ERROR: No Response   
...                                                 ...                 ...   
1722    https://www.thymio.org/sour-gummies-a98662.html  ERROR: No Response   
1723  https://www.universetoday.com/tag/protoplaneta...  ERROR: No Response   
1724  https://www.visionofbritain.org.uk/unit/102274...                NONE   
1725  https://www.wecdsb.on.ca/cms/one.aspx?portalid...  ERROR: No Response   
1726  https://www.wellnessdentalllc.com/for-healthy-...  ERROR: No Response   

                     

In [ ]:
# Drop rows where column 'Error Message' has NaN values
elinks = elinks.dropna(subset=['Error Message'])

print(elinks)

                                                    url         Found Links  \
0     http://150-degree.com/bilder/ebook/download-en...  ERROR: No Response   
1     http://150-degree.com/bilder/ebook/download-pe...  ERROR: No Response   
2     http://3er-schmiede.de/wellrad/plugins/wellrad...  ERROR: No Response   
3     http://automatedgenealogy.com/census/view.jsp?...  ERROR: No Response   
4     http://automatedgenealogy.com/census11/view.js...  ERROR: No Response   
...                                                 ...                 ...   
1721  https://www.thehtgroup.com/ok-boomer-pop-cultu...  ERROR: No Response   
1722    https://www.thymio.org/sour-gummies-a98662.html  ERROR: No Response   
1723  https://www.universetoday.com/tag/protoplaneta...  ERROR: No Response   
1725  https://www.wecdsb.on.ca/cms/one.aspx?portalid...  ERROR: No Response   
1726  https://www.wellnessdentalllc.com/for-healthy-...  ERROR: No Response   

                                          Error Mes

In [ ]:
#filtering by 403 code for forbidden links
elinks = elinks[elinks['Error Message'].str.contains('403')]
print(elinks)

                                                    url         Found Links  \
11    http://doronjo.xsrv.jp/asllrqvcbd/best-rated-c...  ERROR: No Response   
19    http://jamaicaplaingazette.com/2013/07/19/jp-a...  ERROR: No Response   
21    http://preventblindness.org/document/ohios-agi...  ERROR: No Response   
27    http://swamplot.com/pappas-bar-b-q-charred-on-...  ERROR: No Response   
32    http://www.atraditionallifelived.com/2009/10/l...  ERROR: No Response   
...                                                 ...                 ...   
1720  https://www.teachingenglish.org.uk/northern-ir...  ERROR: No Response   
1721  https://www.thehtgroup.com/ok-boomer-pop-cultu...  ERROR: No Response   
1722    https://www.thymio.org/sour-gummies-a98662.html  ERROR: No Response   
1725  https://www.wecdsb.on.ca/cms/one.aspx?portalid...  ERROR: No Response   
1726  https://www.wellnessdentalllc.com/for-healthy-...  ERROR: No Response   

                                          Error Mes

In [ ]:
# Check for duplicates in elinks
# Drop duplicates based on the 'url' column
elinks_cleaned = elinks.drop_duplicates(subset=['url'], keep='first')

# Check for duplicates in elinks
# Drop duplicates based on the 'url' column
errordf_cleaned = errordf.drop_duplicates(subset=['url'], keep='first')

# Merging data sets to bring in content
final_errored_data = pd.merge(elinks_cleaned, errordf_cleaned, on='url', how='left')
print(final_errored_data)


                                                   url       Found Links_x  \
0    http://doronjo.xsrv.jp/asllrqvcbd/best-rated-c...  ERROR: No Response   
1    http://jamaicaplaingazette.com/2013/07/19/jp-a...  ERROR: No Response   
2    http://preventblindness.org/document/ohios-agi...  ERROR: No Response   
3    http://swamplot.com/pappas-bar-b-q-charred-on-...  ERROR: No Response   
4    http://www.atraditionallifelived.com/2009/10/l...  ERROR: No Response   
..                                                 ...                 ...   
693  https://www.teachingenglish.org.uk/northern-ir...  ERROR: No Response   
694  https://www.thehtgroup.com/ok-boomer-pop-cultu...  ERROR: No Response   
695    https://www.thymio.org/sour-gummies-a98662.html  ERROR: No Response   
696  https://www.wecdsb.on.ca/cms/one.aspx?portalid...  ERROR: No Response   
697  https://www.wellnessdentalllc.com/for-healthy-...  ERROR: No Response   

                                         Error Message Found Li

In [ ]:
#Dropping columns to keep URL and Content

final_errored_data = final_errored_data.drop(columns=['Error Message', 'Found Links_x', 'Found Links_y','Keyword'])
print(final_errored_data)

                                                   url  \
0    http://doronjo.xsrv.jp/asllrqvcbd/best-rated-c...   
1    http://jamaicaplaingazette.com/2013/07/19/jp-a...   
2    http://preventblindness.org/document/ohios-agi...   
3    http://swamplot.com/pappas-bar-b-q-charred-on-...   
4    http://www.atraditionallifelived.com/2009/10/l...   
..                                                 ...   
693  https://www.teachingenglish.org.uk/northern-ir...   
694  https://www.thehtgroup.com/ok-boomer-pop-cultu...   
695    https://www.thymio.org/sour-gummies-a98662.html   
696  https://www.wecdsb.on.ca/cms/one.aspx?portalid...   
697  https://www.wellnessdentalllc.com/for-healthy-...   

                                               content  
0    cbd oil pill mixed with ibuprofen best rated c...  
1    jp agenda july 19aug 1 2013  jamaica plain gaz...  
2    ohios aging eye public private partnership  pr...  
3    pappas barbq charred on little york broken poo...  
4    a traditional

In [ ]:
#exporting file
final_errored_data.to_excel('Erroed_Links.xlsx', index=False)